### **1. Importing libraries**

In [ ]:
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cv2

from keras.models import load_model
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from utils.utils import split_data
from crack_segmentation.subroutines.loss_metrics import(
    Weighted_Cross_Entropy,
    F1_score_dil,
    F1_score,
    Precision_dil
)

### **2. Loading the dataset**

In [ ]:
dataset = pd.read_csv('../dataset/dataset_final.csv')

dataset['Filepath'] = dataset['Filepath'].apply(lambda x: '../' + x)

train_df, test_df = train_test_split(
    dataset.sample(6000, random_state=42),
    train_size=0.80,
    random_state=42
)

preprocess_input = tf.keras.applications.resnet.preprocess_input

train_data, valid_data, test_data = split_data(
        train_df, 
        test_df, 
        image_height=224,
        image_width=224
        image_channels=1.0, 
        preprocess_input=preprocess_input,
    )

### **3. Importing the UNet-ResNet50 Model**

In [ ]:
model = load_model(
    './crack_segmentation/output/checkpoints/unet_resnet50.h5',
    custom_objects={
        'loss': Weighted_Cross_Entropy(10),
        'Precision_dil': Precision_dil,
        'F1_score': F1_score,
        'F1_score_dil': F1_score_dil
    }
)

### **4. Evaluating the model**

In [ ]:
loss, acc = model.evaluate(test_data)

print(f"loss' model: {loss}")
print(f"accuracy's model: {acc}")

### **5. Making predictions**

In [ ]:
y_pred = np.argmax(model.predict(test_data), axis=-1)
metrics = classification_report(test_data.labels, y_pred)

print(metrics)